In [1]:
from __future__ import division
from parser import PackToDocks
import codecs 
import os
import glob
from sklearn.cluster import KMeans
import artm
import numpy as np
from itertools import combinations
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
import nltk


/home/gernar/.local/lib/python2.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [11]:
class_list_trump = PackToDocks('trump_labelled.txt', 'data_trump')
class_list_DNR = PackToDocks('lnr_dnr_labelled.txt', 'data_DNR')

In [3]:
def ClearTable(table):
    for i in table:
        if i[0]==i[1]:
            table[i] == False
        else:
            table[i] == 0

def Add(dic, value):
    if value in dic:
        dic[value] +=1
    else:
        dic[value] = 1

def All(dic):
    for i in dic:
        if i[0] == i[1] and dic[i] == False:
            return False
    return True

def GetDict(cluster_list, class_list):
    s = set()
    d = {}
    for x in cluster_list:
        s.add(x)
    for kek in s:
        self_d = {}
        for i in s:
            self_d[i] = 0
        for i in range(len(cluster_list)):
            if cluster_list[i] == kek:
                Add(self_d, class_list[i])
        d[kek] = dict(self_d)
    return d
def Swap(cluster_list, class_list, max_key):
    for i in range(len(cluster_list)):
        if cluster_list[i] == max_key[0]:
            cluster_list[i] = max_key[1]
        elif cluster_list[i] == max_key[1]:
            cluster_list[i] = max_key[0]
def CPS(cluster_list, class_list):
    s = set()
    for x in cluster_list:
        s.add(x)
    table = {}
    for i in combinations(list(s), 2):
        table[i] = 0
    for i in s:
        table[(i, i)] = False
    while True:
        d = GetDict(cluster_list, class_list)
        for line in table:
            if line[0] != line[1]:
                table[line] = d[line[0]][line[1]] + d[line[1]][line[0]]
            else:
                table[line] = d[line[0]][line[1]]
        max_key = None
        for line in table:
            if table[line] > table[(line[0], line[0])] + table[(line[1], line[1])]:
                if max_key == None:
                    max_key = line
                elif table[max_key] < table[line]:
                    max_key = line
        if max_key == None:
            break
        else:
            Swap(cluster_list, class_list, max_key)

    counter = 0
    for i,x in enumerate(class_list):
        if x == cluster_list[i]:
            counter += 1
    return counter/len(class_list)


def PWS(class_list, cluster_list):
    PW = 0
    counter = 0
    for i, x in enumerate(class_list):
        for j, y in enumerate(class_list):
            if x == y:
                counter += 1
                PW += int(cluster_list[i] == cluster_list[j])

    return (PW/counter)


In [4]:
def Clustering(data_path, num_topics = 3, alpha = 0.1, beta = 0.1, 
               num_document_passes = 50, 
               num_collection_passes = 50,
               num_tokens = 10,
              dimension = 5,
              n_clusters = 3):
    file_list = [file for file in os.listdir(data_path) if os.path.isfile(data_path + '/' + file)]
    file_list.sort(key = lambda x: int(x[2:-4]))
    stopword_set = set(stopwords.words('russian') + [u'тот', u'этот', u'он', u'она', u'оно', u'наш', u'ваш', u'это', u'быть'])
    
    files = []
    for j, file in enumerate(file_list):
        with codecs.open(data_path + '/' + file, 'r', 'utf-8') as f:
            sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', f.read())
            with codecs.open('vw.sentences.txt', 'w', 'utf-8') as s:
                for i, sentence in enumerate(sentences):
                    sent_list = nltk.word_tokenize(sentence)
                    sent_list = [sent.lower() for sent in sent_list if (sent.isalpha() and sent.lower() not in stopword_set)]
                    string = u'{0} |@default_class {1}'.format(str(i), ' '.join(sent_list))
                    s.write(string + '\n')
        batch_vectorizer = artm.BatchVectorizer(data_path = 'vw.sentences.txt', 
                                                data_format = 'vowpal_wabbit', 
                                                target_folder = 'TM/batches')
        lda = artm.LDA(dictionary = batch_vectorizer.dictionary, 
                       num_topics = num_topics, 
                       alpha = alpha, 
                       beta = beta, 
                       num_document_passes = num_document_passes)
        lda.fit_offline(batch_vectorizer = batch_vectorizer, num_collection_passes = num_collection_passes)
        
        top_tokens = lda.get_top_tokens(num_tokens = num_tokens)
        
        words = []
        for topic in top_tokens:
            words += topic
        print file + ': ' + u' '.join(words)
        tag = TaggedDocument(words,[j])
        files.append(tag)
        
    vecs = np.zeros(shape = (len(file_list), dimension))
    model = Doc2Vec(files, vector_size=dimension, window=2, min_count=1, workers=4, dm = 1)
    for i in range(len(file_list)):
        vecs[i] = model.docvecs[i]
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(vecs)
    return kmeans.labels_

In [8]:
cluster_list_DNR = Clustering('data_DNR', num_tokens = 1, num_topics = 10, n_clusters = 4)
cluster_list_trump = Clustering('data_trump', num_tokens = 1, num_topics = 10)

d_0.txt: песков народной песков стороны песков песков пытается предприятий государством песков
d_1.txt: ахметова идет шахта днр украинской шахте поясняет донецке предприятия платить
d_2.txt: согласно действий согласно согласно курченко александра этих управления рф капитал
d_3.txt: обстреливали боевики жителей украины боевики новомихайловки населения киндсфатер донецкой обстреливали
d_4.txt: людям украсть людям отжим которые людям людям национализацию людям песков
d_5.txt: предприятий предприятий территориях жителей связи жителей связи днр удалены informnapalm
d_6.txt: млрд действий этих согласно ахметова александра капитал управления рф украине
d_7.txt: эскалации документов украины украины документов рф документов документов шаги документов
d_8.txt: февраля марта долларов днр долларов также сказано ахметова днр группа
d_9.txt: сказал говорится гуманитарной захарченко которые предприятий днр рф днр этих
d_10.txt: участники блокады агрессором марчук украины также области другие тех ново

d_89.txt: лднр россией youtube youtube территориях президент которые наказание youtube порошенко
d_90.txt: своих говорится лнр скм скм предприятий днр предприятий временно поставляем
d_91.txt: целенаправленной документов документов украины документов рф документов документов шаги документов
d_92.txt: фоне великобритании территории территории территории украины территории украинских территории великобритании
d_93.txt: сообщалось боевики также пао дтэк чао ооо министерство ооо днр
d_94.txt: рассказали захарченко перестали марта территории захарченко днр предприятиях голосу блокаду
d_95.txt: элементов предприятий сообщают сообщают которые сообщают блокаду словам регионе песков
d_96.txt: заявили предприятия днр предприятий предприятий днр предприятий поставки предприятий боевиков
d_97.txt: территории боевиков боевиков боевиков боевиков марта боевиков боевиков сепаратистов боевиков
d_98.txt: порошенко который действительно порошенко уголь стабильность решить порошенко украинской украину
d_0

d_82.txt: написал маск своей маск климата выхожу сша маск илон маск
d_83.txt: парижское климата парижского парижское парижское заявлении сша климата целей парижское
d_84.txt: парижское климата парижского парижское парижское заявлении сша климата целей парижское
d_85.txt: климата решение сша парижского сша сша трамп сообщает триумф прюитт
d_86.txt: соглашения сша сша словам соглашение сша трамп рабочих парижское глобальное
d_87.txt: соглашения барака страны парижского парниковых хейли сша обязательства климата трамп
d_88.txt: американским американским американским сша нашу американским трамп американским соглашение американским
d_89.txt: соглашение соглашение соглашение парижского нашу соглашение американским соглашение соглашение сша
d_90.txt: соглашения градуса париже сша долга трамп соглашения время словам трамп
d_91.txt: соглашения градуса париже сша долга трамп соглашения время словам трамп
d_92.txt: соглашения парижского покинуть сша соглашения трампа парижского соглашения маск со

d_178.txt: сша сша сша сша главы правительства макрона страны переговоры президент
d_179.txt: макрон президент мир сша макрон б хотим макрон нашего странице
d_180.txt: мир климата климата климата климата странице хотим климата проблем макрон
d_181.txt: году президент дональд сообщает соглашение трамп сша трампа году назвав
d_182.txt: макрон трампа мир сша повернулись б хотим макрон нашего макрон
d_183.txt: соглашения написала соглашения соглашения ошибка дональда штатов соглашения вперед президент
d_184.txt: сша соглашение парижского сша сша сша трамп трамп сша отметил
d_185.txt: является парникового является вопросам является является является является оон дюжаррик
d_186.txt: парижского новых трамп парижского документ парижская парижского трамп парижского парижского
d_187.txt: актер окружающей сможет трампа шварценеггер шварценеггер бывший неудобная часть парижского
d_188.txt: человек человек человек соглашения путь человек соглашения слишком сша назад
d_189.txt: соглашения позицию со

In [12]:
print 'TRUMP PWS: ' + str(PWS(class_list = class_list_trump, cluster_list = cluster_list_trump))
print 'DNR PWS: ' + str(PWS(class_list = class_list_DNR, cluster_list = cluster_list_DNR))
print '========================================='
print 'TRUMP CPS: ' + str(CPS(class_list = class_list_trump, cluster_list = cluster_list_trump))
print 'DNR CPS: ' + str(CPS(class_list = class_list_DNR, cluster_list = cluster_list_DNR))

TRUMP PWS: 0.342396942397
DNR PWS: 0.262727610554
TRUMP CPS: 0.384615384615
DNR CPS: 0.272727272727


In [13]:
print 'TRUMP: ', class_list_trump
print cluster_list_trump

print 'DNR: ', class_list_DNR
print cluster_list_DNR


TRUMP:  [0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 0, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1]
[1 0 1 0 2 2 2 0 1 0 2 1 2 0 1 2 2 1 0 2 2 0 0 0 1 1 1 2 1 2 1 1 2 1 2 0 2
 2 0 2 1 1 0 0 0 2 0 0 2 2 2 2 0 1 2 0 0 0 1 2 2 1 0 1 0 1 0 1 1 1 2 2 1 0
 0 2 2 2 2 1 2 1 0 1 1 1 0 2 0 1 1 0 1 1 2 0 1 0 1 1 0 1 0 0 1 1 2 1 2 1 1
 1 0 0 0 2 0 2 1 0 2 0 2 1 2 2 2 2 2 2 0 1 1 1 1 0 1 2 2 2 1 0 2 1 1 2 1 2
 1 0 1 1 2 2 0 0 0 1 0 2 2 1

In [ ]:
def Clustering(data_path, num_topics = 3, alpha = 0.1, beta = 0.1, 
               num_document_passes = 50, 
               num_collection_passes = 50,
               num_tokens = 10,
              dimension = 5,
              n_clusters = 3):
    file_list = [file for file in os.listdir(data_path) if os.path.isfile(data_path + '/' + file)]
    file_list.sort(key = lambda x: int(x[2:-4]))
    stopword_set = set(stopwords.words('russian') + [u'тот', u'этот', u'он', u'она', u'оно', u'наш', u'ваш', u'это', u'быть'])
    
    files = []
    for j, file in enumerate(file_list):
        with codecs.open(data_path + '/' + file, 'r', 'utf-8') as f:
            sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', f.read())
            with codecs.open('vw.sentences.txt', 'w', 'utf-8') as s:
                for i, sentence in enumerate(sentences):
                    sent_list = nltk.word_tokenize(sentence)
                    sent_list = [sent.lower() for sent in sent_list if (sent.isalpha() and sent.lower() not in stopword_set)]
                    string = u'{0} |@default_class {1}'.format(str(i), ' '.join(sent_list))
                    s.write(string + '\n')
        batch_vectorizer = artm.BatchVectorizer(data_path = 'vw.sentences.txt', 
                                                data_format = 'vowpal_wabbit', 
                                                target_folder = 'TM/batches')
        lda = artm.LDA(dictionary = batch_vectorizer.dictionary, 
                       num_topics = num_topics, 
                       alpha = alpha, 
                       beta = beta, 
                       num_document_passes = num_document_passes)
        lda.fit_offline(batch_vectorizer = batch_vectorizer, num_collection_passes = num_collection_passes)
        
        top_tokens = lda.get_top_tokens(num_tokens = num_tokens)
        
        words = []
        for topic in top_tokens:
            words += topic
        print file + ': ' + u' '.join(words)
        tag = TaggedDocument(words,[j])
        files.append(tag)
        
    vecs = np.zeros(shape = (len(file_list), dimension))
    model = Doc2Vec(files, vector_size=dimension, window=2, min_count=1, workers=4, dm = 1)
    for i in range(len(file_list)):
        vecs[i] = model.docvecs[i]
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(vecs)
    return kmeans.labels_